# 作業2: ch10 Phthon實作練習(網路開放資料)

## 學號: MB1G0110, 姓名: 陳聖文
---

## 作業說明
本作業使用來自 [政府資料開放平台](https://data.gov.tw/) 的[光觀資訊資料庫](https://data.gov.tw/dataset/7777)  
連結網址: https://data.gov.tw/dataset/7777

## 實作
### 說明
先向政府資料開放平臺的光觀資訊資料庫，以 requests 套件進行請求公開資料集，並以 XML 型式作為快取儲存在本機，以防止過多的請求次數，並增加讀取速度。  
之後以 BeautifulSoup 搭配 lxml 的 XML Parser 進行資料解析，最後以 Pandas 製作為表格顯示，以便於以後利用資料。

### 匯入套件包
使用套件包：
- os
- requests
- pandas
- BeautifulSoup

In [1]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

### 定義全域變數
ROOT_DIR: 根目錄  
XML_FILEPATH: 儲存 XML 檔案的位置  
OPEN_DATASET_URL: 開放資料庫的 API 連結  
DATA_FRAME_COLUMNS: 資料表的表頭。

In [2]:
ROOT_DIR = os.path.abspath(os.getcwd())
XML_FILEPATH = os.path.join(ROOT_DIR, "scenic_spot_C_f.xml")
OPEN_DATASET_URL = "https://gis.taiwan.net.tw/XMLReleaseALL_public/scenic_spot_C_f.xml"
DATA_FRAME_COLUMNS = ["name", "description", "tel", "region", "town", "address"]

### 撰寫相關 Function
為了增加主程式的可讀性，及程式碼的複用性。

#### 請求網路資料
Input: 網址, 編碼(可選)  
Output: 回應結果, 回傳資料

In [3]:
def request(url, encoding="utf-8"):
    res = requests.get(url)
    res.encoding = encoding
    return res, res.text

#### 寫入資料進入檔案
Input: 資料, 檔案路徑, 編碼(可選)  

In [4]:
def save_to_file(data, filepath, encoding="utf-8"):
    with open(filepath, "w", encoding=encoding) as file:
        file.write(data)

#### 載入檔案資料
Input: 檔案路徑, 編碼(可選)  
Output: 資料

In [5]:
def load_from_file(filepath, encoding="utf-8"):
    with open(filepath, "r", encoding=encoding) as file:
        return file.read()

### 請求網路資料
先確認本機快取是否存在，如果不存在則向目標位址請求資料，並儲存到本機作為快取。

In [6]:
if not os.path.exists(XML_FILEPATH):
    response, result = request(OPEN_DATASET_URL)
    # 確認回傳狀態碼是否為，200
    assert response.status_code == requests.codes.ok, f"Request Failed: {response.status_code}"
    soup = BeautifulSoup(result, "xml")
    save_to_file(str(soup), "scenic_spot_C_f.xml")

### 從快取讀取資料
從快取讀取資料，並以 BeautifulSoup 進行解析。

In [7]:
xml_data = load_from_file(XML_FILEPATH)
soup = BeautifulSoup(xml_data, "xml")

### 從 XML 建立 Pandas 資料表
從 XML 中提取資料，建立成 Pandas 資料表，方便於後續資料作業。

In [8]:
# 建立空陣列，來儲存從 XML 中提取出來的資料
tourist_row = []

# 提取出所有的景點資料
tourist_list = soup.find_all("Info")

In [9]:
# 例用 for 迴圈，掃過每一個景點資料
for tourist in tourist_list:
    # 提取出景點資料的特定內容 (名稱, 簡介, 電話, 行政區, 城市, 地址)
    name = tourist.find("Name").text
    description = tourist.find("Toldescribe").text
    tel = tourist.find("Tel").text
    region = tourist.find("Region").text
    town = tourist.find("Town").text
    address = tourist.find("Add").text.replace(region, "").replace(town, "")
    
    # 將提取出的資料，建立成字典形式，並添加進儲存陣列中
    tourist_row.append({
        "name": name,
        "description": description,
        "tel": tel,
        "region": region,
        "town": town,
        "address": address
    })

In [10]:
# 建立 Pandas 資料表
pd_table = pd.DataFrame(tourist_row, columns=DATA_FRAME_COLUMNS)

# 顯示前 10 筆內容
pd_table.head(n=10)

,name,description,tel,region,town,address
0,宏亞食品巧克力觀光工廠,宏亞食品巧克力觀光工廠是一座以巧克力為主題的觀光工廠，建築設計、展場文字、陳列物、戶外景觀及...,886-3-3656555,桃園市,八德區,桃園縣八德市建國路386號
1,台灣金屬創意館,台灣金屬創意館為第一座金屬造型觀光工廠，來這裡可讓大家看到並觸摸到金屬柔軟的一面，館區內充滿...,886-6-2036735#245,臺南市,永康區,台南市永科環路598號
2,臺灣菸酒(股)公司林口觀光酒廠,林口酒廠為擁有近百年歷史的台北酒廠(1921)、板橋酒廠(1937)、樹林酒廠(1906)合...,886-3-3283001#430,桃園市,龜山區,桃園縣龜山鄉文化一路55號
3,水璉、牛山海岸,水璉位在花蓮縣壽豐鄉海濱，蒼翠的山丘環抱著寬廣的河谷盆地，水璉溪蜿蜒而過，沿著公路邊的小徑往...,886-3-8601400,花蓮縣,壽豐鄉,974牛山39之5號
4,石梯坪,石梯坪擁有經風力和海水雕刻而成的特殊岩岸風景，潮間帶上豐富的自然生態資源：螃蟹、海星、海參、...,886-3-8781452,花蓮縣,豐濱鄉,977石梯坪52號
5,長虹橋,長虹橋優雅的拱形橋身橫跨在秀姑巒溪出海口處兩岸，在山海交際之處形成一道美麗的虹影，成為東海岸...,886-3-8671326,花蓮縣,豐濱鄉,977台11線68公里處
6,北回歸線,台灣一共有三個北回歸線標，分別在嘉義水上鄉、花蓮舞鶴台地、豐濱鄉，而這座北回歸線標碑位於台1...,886-3-8671326,花蓮縣,豐濱鄉,977台11線70.5公里處
7,金樽,金樽可俯瞰海岸全景，包含金樽漁港、白沙灣海灘以及附近的山澗峽谷，讓秀麗的景色完全盡收眼底，而...,886-8-9841520,臺東縣,東河鄉,959七里橋11號
8,加路蘭,加路蘭擁有一大片寬廣的草坪、休憩涼亭、解說站以及觀景台，不但提供了東海岸旅行的最佳休息站，這...,886-8-9281530,臺東縣,臺東市,950台11線157公里處
9,富岡地質公園 (小野柳),富岡地質公園 (小野柳) 位於富岡漁港北方的海域，是東部海岸最南端的風景據點，因地形和岩石在...,886-8-9281530,臺東縣,臺東市,950松江路一段500號
